In [2]:
import utils
import plotting
import postprocessing
import numpy as np
import warnings
import pandas as pd
from pandas.errors import SettingWithCopyWarning
from sample_labels import samples, sig_key, data_key

# ignore these because they don't seem to apply
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from PyPDF2 import PdfFileMerger

from copy import deepcopy

import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 16})

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
MAIN_DIR = "../../../"
signal_samples_dir = f"{MAIN_DIR}/../data/skimmer/Feb3"
year = "2017"

# Both Jet's Msds > 50
filters = [
    [
        ("('ak8FatJetMsd', '0')", ">=", 50),
        ("('ak8FatJetMsd', '1')", ">=", 50),
    ],
]

events_dict = utils.load_samples(signal_samples_dir, {sig_key: samples[sig_key]}, year, filters)
tot_weight = np.sum(events_dict[sig_key]["weight"]).values[0]
print(f"Pre-selection {samples[sig_key]} yield: {tot_weight:.2f}")

cutflow = pd.DataFrame(index=[sig_key])


Finding HHbbVV samples
Loading GluGluToHHTobbVV_node_cHHH1
Loaded 236504 entries
Pre-selection GluGluToHHTobbVV yield: 5.12


In [5]:
postprocessing.apply_weights(events_dict, year, cutflow)
bb_masks = postprocessing.bb_VV_assignment(events_dict)[sig_key]
events = events_dict[sig_key]

In [6]:
postprocessing.postprocess_lpsfs(events)

/Users/raghav/Documents/CERN/hhbbww/HHbbVV/src/HHbbVV/postprocessing/postprocessing.py:339: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  events[[(f"{jet}_{key}", i) for i in range(num_lp_sf_toys)]] = td[key]
/Users/raghav/Documents/CERN/hhbbww/HHbbVV/src/HHbbVV/postprocessing/postprocessing.py:339: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  events[[(f"{jet}_{key}", i) for i in range(num_lp_sf_toys)]] = td[key]


In [23]:
VV_cut_events = events[utils.get_feat(events, "VVFatJetParTMD_THWW4q", bb_masks) > 0.975]
VV_pt_cut_events = events[(utils.get_feat(events, "VVFatJetParTMD_THWW4q", bb_masks) > 0.975) * (utils.get_feat(events, "VVFatJetPt", bb_masks) > 500)]
bb_cut_events = events[utils.get_feat(events, "bbFatJetParticleNetMD_Txbb", bb_masks) > 0.975]

In [10]:
vvsf, uncs = postprocessing.get_lpsf(VV_cut_events)
tot_rel_unc = np.linalg.norm([val for val in uncs.values()])
tot_unc = vvsf * tot_rel_unc

print(f"SF: {vvsf:.2f} ± {tot_unc:.2f}")
for key, val in uncs.items():
    print(f"{key}: {val:.2f}")
print(f"Combined: {tot_rel_unc:.2f}")

SF: 0.77 ± 0.23
syst_unc: 0.28
stat_unc: 0.04
sj_pt_unc: 0.00
sj_matching_unc: 0.12
Combined: 0.30


In [24]:
vvsf, uncs = postprocessing.get_lpsf(VV_pt_cut_events)
tot_rel_unc = np.linalg.norm([val for val in uncs.values()])
tot_unc = vvsf * tot_rel_unc

print(f"SF: {vvsf:.2f} ± {tot_unc:.2f}")
for key, val in uncs.items():
    print(f"{key}: {val:.2f}")
print(f"Combined: {tot_rel_unc:.2f}")

SF: 0.73 ± 0.33
syst_unc: 0.43
stat_unc: 0.07
sj_pt_unc: 0.02
sj_matching_unc: 0.10
Combined: 0.45


In [20]:
vvsf, uncs = postprocessing.get_lpsf(bb_cut_events, VV=False)
tot_rel_unc = np.linalg.norm([val for val in uncs.values()])
tot_unc = vvsf * tot_rel_unc

print(f"SF: {vvsf:.2f} ± {tot_unc:.2f}")
for key, val in uncs.items():
    print(f"{key}: {val:.2f}")
print(f"Combined: {tot_rel_unc:.2f}")

SF: 0.91 ± 0.35
syst_unc: 0.38
stat_unc: 0.04
sj_pt_unc: 0.02
sj_matching_unc: 0.03
Combined: 0.38
